In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units
    
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/datashotiqr/LW.csv'
df = pd.read_csv(url, delimiter=',')
# Menentukan input(X) dan output (y)
X = df.drop(columns=["SHOT","GOAL","7MACC","6MACC","9MACC","WINGACC","PIVOTACC","FASTBREAKACC"])
y = df.GOAL
# Membagi dataset 80:20 untuk train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# normalisasi dataset agar tidak bias
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

validation_results_df = pd.DataFrame(columns=['R2_Test', 'MSE', 'MAE'])

for _ in range(20):
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='rmsprop', loss=mean_squared_error, metrics=['mean_absolute_error']) 
    history = model.fit(X_train, y_train, batch_size=8, epochs=100, validation_data=(X_test, y_test))
    
    pred_test = model.predict(X_test)
    
    r2_test = r2_score(y_test, pred_test)
    mse = history.history['val_loss'][-1]  # Last MSE from validation data
    mae = history.history['val_mean_absolute_error'][-1]  # Last MAE from validation data
    
    print(f"R-Squared Test Score: {r2_test}")
    
    # Append results to the validation results DataFrame
    validation_results_df = validation_results_df.append({'R2_Test': r2_test,
                                                          'MSE': mse,
                                                          'MAE': mae}, ignore_index=True)
    
    # Reset the TensorFlow session to clear the model
    tf.keras.backend.clear_session()

# Save the validation results DataFrame to a CSV file
validation_results_df.to_csv('validation_resultsshot.csv', index=False)


Epoch 1/100
233/233 [==============================] - 1s 2ms/step - loss: 10.2864 - mean_absolute_error: 2.2746 - val_loss: 8.2771 - val_mean_absolute_error: 2.0167
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 6.1136 - mean_absolute_error: 1.8223 - val_loss: 6.1066 - val_mean_absolute_error: 1.8964
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 5.1202 - mean_absolute_error: 1.7674 - val_loss: 5.4416 - val_mean_absolute_error: 1.8177
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 4.6258 - mean_absolute_error: 1.6961 - val_loss: 5.0099 - val_mean_absolute_error: 1.7400
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 4.2724 - mean_absolute_error: 1.6338 - val_loss: 4.6640 - val_mean_absolute_error: 1.6808
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 3.9764 - mean_absolute_error: 1.5819 - val_loss: 4.3739 - val_mean_absolute_error: 1.6305
Epoch 7/1

C:\Users\Wandi\AppData\Local\Temp\ipykernel_145428\482172737.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


233/233 [==============================] - 1s 2ms/step - loss: 6.2531 - mean_absolute_error: 1.8473 - val_loss: 6.1974 - val_mean_absolute_error: 1.9125
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 5.1761 - mean_absolute_error: 1.7710 - val_loss: 5.5725 - val_mean_absolute_error: 1.8371
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 4.7459 - mean_absolute_error: 1.7080 - val_loss: 5.2020 - val_mean_absolute_error: 1.7784
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 4.4383 - mean_absolute_error: 1.6600 - val_loss: 4.9006 - val_mean_absolute_error: 1.7273
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 4.1757 - mean_absolute_error: 1.6144 - val_loss: 4.6254 - val_mean_absolute_error: 1.6908
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 3.9549 - mean_absolute_error: 1.5828 - val_loss: 4.4301 - val_mean_absolute_error: 1.6347
Epoch 7/100
233/233 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_145428\482172737.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


233/233 [==============================] - 1s 2ms/step - loss: 8.1654 - mean_absolute_error: 2.0043 - val_loss: 7.1436 - val_mean_absolute_error: 1.9041
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 5.5727 - mean_absolute_error: 1.7927 - val_loss: 6.0292 - val_mean_absolute_error: 1.8977
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 5.1728 - mean_absolute_error: 1.7823 - val_loss: 5.7303 - val_mean_absolute_error: 1.8686
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 4.9913 - mean_absolute_error: 1.7615 - val_loss: 5.5357 - val_mean_absolute_error: 1.8295
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 4.8351 - mean_absolute_error: 1.7296 - val_loss: 5.3450 - val_mean_absolute_error: 1.7973
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 4.6800 - mean_absolute_error: 1.7022 - val_loss: 5.1641 - val_mean_absolute_error: 1.7672
Epoch 7/100
233/233 [=

...

C:\Users\Wandi\AppData\Local\Temp\ipykernel_145428\482172737.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


233/233 [==============================] - 1s 2ms/step - loss: 7.6287 - mean_absolute_error: 1.9283 - val_loss: 6.6348 - val_mean_absolute_error: 1.9180
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 5.2133 - mean_absolute_error: 1.7579 - val_loss: 5.4295 - val_mean_absolute_error: 1.7880
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 4.5335 - mean_absolute_error: 1.6609 - val_loss: 4.8887 - val_mean_absolute_error: 1.6996
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 4.1532 - mean_absolute_error: 1.5937 - val_loss: 4.5354 - val_mean_absolute_error: 1.6544
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 3.8871 - mean_absolute_error: 1.5526 - val_loss: 4.2778 - val_mean_absolute_error: 1.6003
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 3.6792 - mean_absolute_error: 1.5116 - val_loss: 4.0556 - val_mean_absolute_error: 1.5666
Epoch 7/100
233/233 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_145428\482172737.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


233/233 [==============================] - 1s 2ms/step - loss: 9.8090 - mean_absolute_error: 2.2420 - val_loss: 8.0827 - val_mean_absolute_error: 1.9947
Epoch 2/100
233/233 [==============================] - 0s 1ms/step - loss: 5.9661 - mean_absolute_error: 1.7902 - val_loss: 6.0846 - val_mean_absolute_error: 1.8762
Epoch 3/100
233/233 [==============================] - 0s 1ms/step - loss: 5.1063 - mean_absolute_error: 1.7582 - val_loss: 5.5693 - val_mean_absolute_error: 1.8320
Epoch 4/100
233/233 [==============================] - 0s 1ms/step - loss: 4.8152 - mean_absolute_error: 1.7272 - val_loss: 5.3077 - val_mean_absolute_error: 1.7914
Epoch 5/100
233/233 [==============================] - 0s 1ms/step - loss: 4.6009 - mean_absolute_error: 1.6876 - val_loss: 5.0684 - val_mean_absolute_error: 1.7613
Epoch 6/100
233/233 [==============================] - 0s 1ms/step - loss: 4.4036 - mean_absolute_error: 1.6591 - val_loss: 4.8555 - val_mean_absolute_error: 1.7196
Epoch 7/100
233/233 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_145428\482172737.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,
